# Qwen3-1.7B GSM8K PPO Training (VERL)

Replicates reported improvement on GSM8K:
| Model | Accuracy |
|---|---|
| Qwen3-1.7B base | ~69.2% |
| Qwen3-1.7B + PPO | ~82.7% |

**GPU recommendation:** A100 40GB (Colab Pro) or T4 16GB (free tier, with reduced batch sizes)

**Runtime:** ~6–12 hours on A100 for 500 steps. Use Colab Pro with background execution.

## 0. Check GPU

In [ ]:
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name}  VRAM: {gpu.total_memory / 1e9:.1f} GB")

## 1. Mount Google Drive (saves checkpoints + logs — survives disconnects)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
DRIVE_DIR = '/content/drive/MyDrive/LLM_RL_runs'
os.makedirs(DRIVE_DIR, exist_ok=True)
print(f'Drive mounted. All checkpoints and logs will be saved to: {DRIVE_DIR}')

## 2. Clone Project Repo

In [ ]:
import os

REPO_URL = "https://github.com/jiayiderekchen/LLM_RL.git"
REPO_DIR = "/content/LLM_RL"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    !git -C {REPO_DIR} pull

os.chdir(REPO_DIR)
!ls

## 2. Install Dependencies

In [ ]:
# Install VERL
!git clone https://github.com/verl-project/verl /content/verl
%cd /content/verl
!pip install -e . -q
%cd {REPO_DIR}

In [ ]:
# Install remaining dependencies
!pip install -q \
    vllm \
    datasets \
    sympy \
    regex \
    pandas \
    pyarrow \
    transformers>=4.44.0 \
    accelerate \
    flash-attn --no-build-isolation

print("Dependencies installed.")

## 3. HuggingFace Login (required to download Qwen3)

In [ ]:
from huggingface_hub import login
# Get your token from https://huggingface.co/settings/tokens
# Use Colab Secrets (key icon on left sidebar) → add HF_TOKEN
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Logged in via Colab secret.")
except Exception:
    login()  # interactive prompt

## 4. Prepare GSM8K Dataset

In [ ]:
os.chdir(REPO_DIR)
!python data/prepare_gsm8k.py --output_dir data/gsm8k
!ls data/gsm8k/

## 5. Test Reward Function

In [ ]:
!python rewards/gsm8k_reward.py

## 6. (Optional) Evaluate Base Model Before Training

This gives you the ~69% baseline to compare against after PPO.

In [ ]:
# Evaluate base model (takes ~30-45 min on T4, ~15 min on A100)
# You can skip this and come back after training.
RUN_BASE_EVAL = False  # Set to True to run

if RUN_BASE_EVAL:
    !python evaluation/eval_gsm8k.py \
        --model_path Qwen/Qwen3-1.7B \
        --split test \
        --max_new_tokens 512 \
        --batch_size 16

## 7. Detect GPU and Configure Training

- **A100 (40GB)**: uses full config, batch_size=128
- **T4 (16GB)**: uses reduced config, batch_size=32, response_length=512

In [ ]:
import torch
import yaml

gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
print(f"GPU memory: {gpu_mem_gb:.1f} GB")

IS_A100 = gpu_mem_gb >= 38
print(f"Using {'A100' if IS_A100 else 'T4/low-VRAM'} config")

In [ ]:
# Write a Colab-specific override config
import os

os.makedirs("configs", exist_ok=True)

if IS_A100:
    colab_overrides = """
data:
  train_batch_size: 128
  max_response_length: 1024

actor_rollout_ref:
  actor:
    ppo_mini_batch_size: 64
    ppo_micro_batch_size_per_gpu: 8
  rollout:
    response_length: 1024
    gpu_memory_utilization: 0.45
  ref:
    fsdp_config:
      param_offload: True

critic:
  ppo_micro_batch_size_per_gpu: 8

trainer:
  n_gpus_per_node: 1
  total_epochs: 15
  save_freq: 50
  test_freq: 25
"""
else:
    # T4 16GB — reduce everything
    colab_overrides = """
data:
  train_batch_size: 32
  max_prompt_length: 384
  max_response_length: 512

actor_rollout_ref:
  model:
    enable_gradient_checkpointing: True
    fsdp_config:
      param_offload: True
  actor:
    ppo_mini_batch_size: 16
    ppo_micro_batch_size_per_gpu: 2
  rollout:
    response_length: 512
    gpu_memory_utilization: 0.35
  ref:
    fsdp_config:
      param_offload: True

critic:
  ppo_micro_batch_size_per_gpu: 2
  model:
    enable_gradient_checkpointing: True
    fsdp_config:
      param_offload: True

trainer:
  n_gpus_per_node: 1
  total_epochs: 10
  save_freq: 50
  test_freq: 50
"""

with open("configs/colab_overrides.yaml", "w") as f:
    f.write(colab_overrides)
print("Config written.")
print(colab_overrides)

## 8. WandB Login (for live training dashboard)

Free at [wandb.ai](https://wandb.ai). After training starts, a dashboard URL will print — open it to see live reward, loss, and accuracy curves.

Skip this cell if you don't want WandB (training still runs, just no dashboard).

In [ ]:
!pip install -q wandb

import wandb
# Add WANDB_API_KEY to Colab Secrets (key icon), or login interactively below
try:
    from google.colab import userdata
    wandb.login(key=userdata.get('WANDB_API_KEY'))
    print('Logged in via Colab secret.')
except Exception:
    wandb.login()  # interactive — paste your API key from wandb.ai/authorize

## 9. PPO Training

> **Important:** Enable Colab background execution before running:
> Runtime → Change runtime type → enable **Background execution**.
> This prevents the session from disconnecting during the long training run.

Progress is visible in two ways:
- **WandB dashboard** (live link printed when training starts)
- **Log monitor cell below** — run it in parallel to tail the log file

In [ ]:
import os
os.chdir(REPO_DIR)
os.environ['PYTHONPATH'] = REPO_DIR

# Run training — custom_reward_function is a valid VERL config key, no + needed
!python -m verl.trainer.main_ppo \
    --config-path {REPO_DIR}/configs \
    --config-name qwen3_gsm8k_ppo \
    trainer.n_gpus_per_node=1 \
    data.train_files={REPO_DIR}/data/gsm8k/train.parquet \
    data.val_files={REPO_DIR}/data/gsm8k/test.parquet \
    +custom_reward_function.path={REPO_DIR}/rewards/gsm8k_reward.py \
    +custom_reward_function.name=compute_score \
    trainer.default_local_dir={REPO_DIR}/checkpoints \
    trainer.experiment_name=qwen3_gsm8k_ppo_colab \
    trainer.project_name=gsm8k_ppo 2>&1 | tee {REPO_DIR}/training.log

### 9b. (Optional) Live Log Monitor
Run this in a **separate cell** while training is running to tail key metrics.

In [ ]:
import time, re, os
from IPython.display import clear_output

LOG_FILE = f"{REPO_DIR}/training.log"
POLL_INTERVAL = 30  # seconds between refreshes

step_pattern = re.compile(r'global_step[:\s]+(\d+)')
reward_pattern = re.compile(r'reward[:\s/]+([\d\.]+)', re.IGNORECASE)
loss_pattern = re.compile(r'(?:actor_loss|loss)[:\s]+([\d\.\-]+)', re.IGNORECASE)

print(f"Monitoring {LOG_FILE} — refreshes every {POLL_INTERVAL}s. Interrupt to stop.")

history = []  # (step, reward)

while True:
    if not os.path.exists(LOG_FILE):
        print("Waiting for training.log to appear...")
        time.sleep(POLL_INTERVAL)
        continue

    with open(LOG_FILE) as f:
        lines = f.readlines()

    # Parse last 200 lines for metrics
    recent = lines[-200:]
    steps = [int(m.group(1)) for l in recent if (m := step_pattern.search(l))]
    rewards = [float(m.group(1)) for l in recent if (m := reward_pattern.search(l))]

    clear_output(wait=True)
    print(f"=== Training Monitor  [{time.strftime('%H:%M:%S')}] ===")
    print(f"Log lines so far : {len(lines)}")
    print(f"Latest step      : {steps[-1] if steps else 'N/A'}")
    print(f"Latest reward    : {rewards[-1]:.4f}" if rewards else "Latest reward    : N/A")
    print(f"Mean reward (recent 10): {sum(rewards[-10:])/len(rewards[-10:]):.4f}" if len(rewards) >= 2 else "")
    print()
    print("--- Last 10 log lines ---")
    for l in lines[-10:]:
        print(l.rstrip())

    time.sleep(POLL_INTERVAL)

## 9. Evaluate Trained Checkpoint

In [ ]:
import glob

# Find latest checkpoint
checkpoints = sorted(glob.glob(f"{REPO_DIR}/checkpoints/qwen3_gsm8k_ppo_colab/global_step_*"))
if checkpoints:
    latest_ckpt = checkpoints[-1]
    print(f"Latest checkpoint: {latest_ckpt}")
else:
    print("No checkpoints found yet.")
    latest_ckpt = "Qwen/Qwen3-1.7B"  # fallback to base

In [ ]:
os.chdir(REPO_DIR)
!python evaluation/eval_gsm8k.py \
    --model_path {latest_ckpt} \
    --split test \
    --max_new_tokens 512

## 10. Save Results to Google Drive (optional)

In [ ]:
SAVE_TO_DRIVE = False  # Set True to mount Drive and copy results

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_DIR = "/content/drive/MyDrive/LLM_RL_results"
    !mkdir -p {DRIVE_DIR}
    !cp -r {REPO_DIR}/checkpoints {DRIVE_DIR}/
    !cp -r {REPO_DIR}/evaluation/results {DRIVE_DIR}/
    !cp {REPO_DIR}/training.log {DRIVE_DIR}/
    print(f"Saved to {DRIVE_DIR}")

## 11. Print Accuracy Report

In [ ]:
import json, glob

result_files = glob.glob(f"{REPO_DIR}/evaluation/results/*.json")
for rf in sorted(result_files):
    with open(rf) as f:
        data = json.load(f)
    r = data["report"]
    print(f"{r['model_path']}  →  {r['accuracy']*100:.2f}%  ({r['correct']}/{r['total']})")